# part1

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.initializers import RandomUniform

In [4]:
from tensorflow.python.keras.models import Sequential
#from tensorflow.keras.layers import Dense,LSTM,Dropout

In [5]:
#from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding,LSTM,Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [113]:
GivenData = pd.read_csv('GoogleStocks.csv')

In [114]:
GivenData = GivenData.iloc[::-1]

In [115]:
GivenData['avgHighLow'] = (GivenData['high']+GivenData['low'])/2

In [116]:
ConsideredGivenData  = GivenData[['date','avgHighLow','volume','open']]

In [117]:
ConsideredGivenDataShifted = ConsideredGivenData.shift(-1)

In [118]:
Xdata = ConsideredGivenData[['avgHighLow','volume']].values[0:-1]#,'open']]

In [11]:
#Xdata = ConsideredGivenData[['avgHighLow','volume','open']]#.values[0:-1]

In [119]:
Ydata = ConsideredGivenDataShifted['open'].values[:-1]

In [120]:
print(type(Xdata))
print(Xdata.shape)
#print(type(Ydata))
#print(Ydata.shape)
Ydata = Ydata.reshape(len(Ydata),1)
print(Ydata.shape)

<class 'numpy.ndarray'>
(754, 2)
(754, 1)


In [121]:
Xdata

array([[7.35745000e+02, 1.30081700e+06],
       [7.38005000e+02, 1.90225400e+06],
       [7.53310000e+02, 1.78227800e+06],
       ...,
       [1.22247000e+03, 1.19589900e+06],
       [1.21641250e+03, 1.66891000e+06],
       [1.19671875e+03, 1.49384100e+06]])

In [122]:
Xdata = np.c_[Xdata,Ydata]

In [123]:
Xdata

array([[7.35745000e+02, 1.30081700e+06, 7.34590000e+02],
       [7.38005000e+02, 1.90225400e+06, 7.50100000e+02],
       [7.53310000e+02, 1.78227800e+06, 7.49250000e+02],
       ...,
       [1.22247000e+03, 1.19589900e+06, 1.22632000e+03],
       [1.21641250e+03, 1.66891000e+06, 1.19693000e+03],
       [1.19671875e+03, 1.49384100e+06, 1.19853000e+03]])

In [124]:
Xtrain = Xdata[0:int(0.8*len(Xdata))]
Xtest = Xdata[int(0.8*len(Xdata)):]
#Ytrain = Ydata[0:int(0.8*len(Xdata))]
#Ytest = Ydata[int(0.8*len(Xdata)):]

In [125]:
len(Xtrain)

603

In [126]:
len(Xtest)

151

In [127]:
Xscaler = MinMaxScaler()
XtrainScaled = Xscaler.fit_transform(Xtrain)
XtestScaled = Xscaler.transform(Xtest)

In [128]:
XtestScaled

array([[0.92373201, 0.14465767, 0.88505   ],
       [0.89728701, 0.15606399, 0.89003333],
       [0.90435796, 0.06049953, 0.895     ],
       [0.91448916, 0.12025193, 0.88166667],
       [0.90876429, 0.06364048, 0.89356667],
       [0.92117125, 0.08341729, 0.89636667],
       [0.92327469, 0.07566146, 0.92766667],
       [0.95775489, 0.11416957, 0.95048333],
       [0.95980778, 0.14032471, 0.94408333],
       [0.97314763, 0.14021516, 0.95538333],
       [0.97247674, 0.14470866, 0.93996667],
       [0.94181465, 0.23562534, 0.88878333],
       [0.90424268, 0.23343476, 0.87133333],
       [0.86664959, 0.27837034, 0.85883333],
       [0.84745632, 0.24935674, 0.81278333],
       [0.84253326, 0.15963152, 0.83531667],
       [0.84437149, 0.10683442, 0.81771667],
       [0.84459968, 0.12418472, 0.8362    ],
       [0.84633226, 0.13935884, 0.8329    ],
       [0.85012703, 0.15981091, 0.84683333],
       [0.85629206, 0.07340214, 0.8319    ],
       [0.84151484, 0.13711283, 0.81015   ],
       [0.

In [130]:
XtestScaled[10:30,2]

array([0.93996667, 0.88878333, 0.87133333, 0.85883333, 0.81278333,
       0.83531667, 0.81771667, 0.8362    , 0.8329    , 0.84683333,
       0.8319    , 0.81015   , 0.8233    , 0.84831667, 0.86833333,
       0.81028333, 0.84191667, 0.85691667, 0.85955   , 0.86811667])

In [150]:
from hmmlearn.hmm import GaussianHMM
from hmmlearn.hmm import GMMHMM

In [132]:
n_hidden_states = 3
n_latency_days = 10

In [155]:
hmm1 = GaussianHMM(n_components=n_hidden_states,covariance_type="full")

In [156]:
feature_vector = np.column_stack((XtrainScaled[:,0], XtrainScaled[:,1], XtrainScaled[:,2]))

In [153]:
hmm1.n_features = 3

In [161]:
hmm1.fit(feature_vector)

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=3, n_iter=10, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [159]:
len(hmm1.predict(feature_vector))

603

In [140]:
hmm1.monitor_.converged

True

In [171]:
np.mean(hmm1.sample(1)[0],axis = 0)

array([0.09900965, 0.18928405, 0.08725144])

In [141]:
from itertools import product
AvgHighLowStock = np.linspace(0.1, 1, 10)
VolumeOfStock = np.linspace(0.1, 1, 10)
openingStock = np.linspace(0.1, 1, 10)
possible_outcomes = np.array(list(product(AvgHighLowStock, VolumeOfStock, openingStock)))

In [75]:
possible_outcomes = hmm1.sample(1000)[0]

In [142]:
possible_outcomes

array([[0.1, 0.1, 0.1],
       [0.1, 0.1, 0.2],
       [0.1, 0.1, 0.3],
       ...,
       [1. , 1. , 0.8],
       [1. , 1. , 0.9],
       [1. , 1. , 1. ]])

In [236]:
def _get_most_probable_outcome(day_index):
    previous_data_start_index = max(0, day_index - n_latency_days)
    print(n_latency_days)
    previous_data_end_index = max(0, day_index - 1)
    previous_data_features = XtestScaled[previous_data_start_index:previous_data_end_index,:]
    #previous_data = self._test_data.iloc[previous_data_end_index: previous_data_start_index]
    #previous_data_features = StockPredictor._extract_features(previous_data)

    outcome_score = []
    for possible_outcome in possible_outcomes:
        total_data = np.row_stack((previous_data_features, possible_outcome))
        outcome_score.append(hmm1.score(total_data))
    most_probable_outcome = possible_outcomes[np.argmax(outcome_score)] 
    return most_probable_outcome
 
def predict_open_price(day_index):
    #open_price = self._test_data.iloc[day_index]['open']
    _, _,predicted_open_price = _get_most_probable_outcome(day_index)
    return predicted_open_price
    #return open_price * (1 + predicted_frac_change)

In [61]:
 def predict_open_prices_for_days(days, with_plot=True):
        predicted_open_prices = []
        for day_index in tqdm(range(20,days)):
            predicted_open_prices.append(predict_open_price(day_index))
        #predicted_open_prices = Xscaler.inverse_transform(predicted_open_prices)
 
        if with_plot:
            test_data = XtestScaled[20:days,2]
            plt.figure(figsize=(15,5))
            plt.plot(test_data, label='true')
            plt.plot(predicted_open_prices, label='pred')
            #test_data = self._test_data[0: days]
            #days = np.array(test_data['date'], dtype="datetime64[ms]")
            #actual_close_prices = test_data['close']
 
            #fig = plt.figure()
 
            #axes = fig.add_subplot(111)
            #axes.plot(days, actual_close_prices, 'bo-', label="actual")
            #axes.plot(days, predicted_close_prices, 'r+-', label="predicted")
            #axes.set_title('{company}'.format(company=self.company))
 
            #fig.autofmt_xdate()
 
            plt.legend()
            plt.show()
 
        return predicted_open_prices

In [237]:
predict_open_prices_for_days(50,True)

  0%|                                                   | 0/30 [00:00<?, ?it/s]

10


  3%|█▍                                         | 1/30 [00:01<00:29,  1.03s/it]

10


  7%|██▊                                        | 2/30 [00:02<00:28,  1.02s/it]

...

 27%|███████████▍                               | 8/30 [00:08<00:23,  1.05s/it]

10


 30%|████████████▉                              | 9/30 [00:09<00:22,  1.05s/it]

10


KeyboardInterrupt: 

In [73]:
0.32342537767493024 in possible_outcomes

True

In [71]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24],[10]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]
X3 = [[0.5,2.4,6.8],[0.56,2.47,6.89],[0.53,2.41,6.82]]

In [72]:
 X = np.concatenate([X1, X2])
lengths = [len(X1), len(X2)]

In [91]:
remodel = hmm.GaussianHMM(n_components=3)
remodel.n_features = 3
remodel.fit(X3)

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=3, n_iter=10, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [90]:
Z2 = remodel.predict(X3)

ValueError: rows of transmat_ must sum to 1.0 (got [1. 1. 0.])

In [83]:
X5, Z = remodel.sample(4)

In [84]:
X5

array([[0.61941627, 2.58958985, 6.99528647],
       [0.45104272, 2.49137542, 6.94093741],
       [0.64840139, 2.19530381, 6.81781214],
       [0.40363441, 2.48942959, 6.86178877]])

In [69]:
remodel = hmm.GaussianHMM(n_components=3)
remodel.fit(X,lengths)
#GaussianHMM(algorithm='viterbi',...
Z2 = remodel.predict(X,lengths)
X5, Z = remodel.sample(4)


ValueError: rows of transmat_ must sum to 1.0 (got [1. 0. 1.])

In [61]:
X5

array([[ 0.62334716],
       [ 0.11387577],
       [ 0.36535789],
       [-0.16088644]])

In [44]:
hmm.GaussianHMM(n_components=3).fit(X, lengths)  

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=3, n_iter=10, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [50]:
remodel.monitor_  

ConvergenceMonitor(history=[-11.664093027023965, -11.663370402922695], iter=9,
          n_iter=100, tol=0.01, verbose=False)

In [51]:
remodel.monitor_.converged

True

In [30]:
from tqdm import tqdm
for hello in tqdm(range(10,20)):
    pass
    #print(hello)    

100%|███████████████████████████████████████| 10/10 [00:00<00:00, 10140.97it/s]


In [233]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]
X1 = np.array(X1)
X2 = np.array(X2)
X = np.concatenate([X1, X2])
X = np.array(X)
lengths = [len(X1), len(X2)]
hmm2 = GaussianHMM(n_components=3, covariance_type='full').fit(X, lengths)  
hmm2.n_features = 4
hmm2.score(X)

-28.279831607952705

In [235]:
hmm2.sample(10)

(array([[ 3.31134698],
        [ 0.48736837],
        [-1.28788378],
        [ 0.55857344],
        [ 0.48052478],
        [-1.66902064],
        [ 0.05407992],
        [ 1.6179776 ],
        [ 0.40867493],
        [ 0.40535045]]), array([1, 2, 0, 2, 2, 0, 0, 0, 2, 0]))

In [187]:
list(np.array(X1))

[array([0.5]), array([1.]), array([-1.]), array([0.42]), array([0.24])]

In [205]:
hmm2.score(X)

-28.279831607952673